In [1]:
import glob
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from collections import OrderedDict
import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from sklearn.metrics import log_loss

In [5]:
sys.path.append('../..')
from helpers.metrics import brier_multi, CalibrationErrors, classification_error
from helpers.settings import models_folder

## Load Data

In [6]:
from keras.datasets import mnist
from keras.utils import to_categorical

def load_mnist():
    num_classes = 10
    img_rows, img_cols = 28, 28

    (x_train_val, y_train_val), (x_test, y_test) = mnist.load_data()
    x_train_val = x_train_val.reshape(x_train_val.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)



    x_train_val = x_train_val.astype('float32')
    x_test = x_test.astype('float32')

    x_train_val /= 255
    x_test /= 255

    y_train_val = to_categorical(y_train_val, num_classes)
    y_test = to_categorical(y_test, num_classes)

    x_train = x_train_val[:50000]
    y_train = y_train_val[:50000]
    x_val = x_train_val[50000:]
    y_val = y_train_val[50000:]
    return (x_train, y_train), (x_val, y_val), (x_test, y_test)

In [7]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = load_mnist()

## Model
* The model is similar to the one that was used in Deep Ensembles and XXX paper
* TODO: try a version of the model without batch normalization and study the effect

In [8]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.layers.core import Lambda
from keras.layers import Conv2D, MaxPooling2D, Flatten

from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.optimizers import Adam
from keras import backend as K

In [9]:
def cnn(num_classes=10):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(BatchNormalization(momentum=.9))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [10]:
n_models = 25
epochs = 15
learning_rate = 2.5e-4
output_folder = os.path.join(models_folder, 'mnist_cnn')
if not os.path.isdir(output_folder):
    os.makedirs(output_folder)
#
output_baseline_folder = os.path.join(output_folder, 'baselines')
if not os.path.isdir(output_baseline_folder):
    os.makedirs(output_baseline_folder)

In [11]:
for i in range(n_models):
    seed = i+17
    # change the random state
    np.random.seed(seed)
    # creating output folder for each of the baseline models
    output_model_folder = os.path.join(output_baseline_folder, str(seed))
    if not os.path.isdir(output_model_folder):
        os.mkdir(output_model_folder)
    else:
        print('skip training...')
        continue
    # callbacks for training
    callbacks = list()
    log = CSVLogger(os.path.join(output_model_folder, 'log.csv'))
    callbacks.append(log)
    checkpoint_path = os.path.join(output_model_folder, 'weights.{epoch:02d}.hdf5')
    checkpoint = ModelCheckpoint(checkpoint_path, period=1, save_best_only=False, verbose=True)
    callbacks.append(checkpoint)
    # initialize and compile the model
    model = cnn()
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate), metrics=['accuracy'])
    # train
    model.fit(x_train, y_train, batch_size=128, epochs=epochs, verbose=1, callbacks=callbacks, validation_data=(x_val, y_val), shuffle=True)

skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...
skip training...


In [ ]:
for i in range(17, 17+n_models):
    log = pd.read_csv(os.path.join(output_baseline_folder, str(i), 'log.csv'))
    plt.figure(figsize=(8,2),dpi=100)
    plt.subplots_adjust(wspace=0.5)
    ax1 = plt.subplot(121)
    ax2 = plt.subplot(122)
    for metric_type in ['', 'val_']:
        for key in log.keys():
            if 'loss' in key and 'val' not in key:
                p = ax1.plot(log[metric_type+ key], label=metric_type + key+str(i), linewidth=1)
                c = p[-1].get_color()
                ax1.plot(np.argmin(log[metric_type+key]), np.amin(log[metric_type+key]), 
                         '*', markersize=10, alpha=0.5, color = c)
                ax1.set_title('min val NLL: {0:.3f}'.format(np.amin(log['val_' + key])))
                ax1.set_ylabel('NLL')
                ax1.set_xlabel('epochs')
            if 'acc' in key and 'val' not in key:
                p = ax2.plot(log[metric_type + key], label=metric_type + key+str(i), linewidth=1)
                c = p[-1].get_color()
                ax2.plot(np.argmax(log[metric_type + key]), np.amax(log[metric_type + key]), 
                         '*', markersize=10, alpha=0.5, color=c)
                ax2.set_title('max val acc: {0:.2f}'.format(np.amax(log['val_' + key]*100)))
                ax2.set_ylabel('accuracy')
                ax2.set_xlabel('epochs')
    ax1.grid()
    ax2.grid()
    ax1.legend()
    ax2.legend()

In [12]:
for index, model_id in enumerate(range(17, 17+n_models)):
    model_folder = os.path.join(output_baseline_folder, str(model_id))
    test_pred_path = os.path.join(model_folder, 'test_pred.npy')
    # if True:
    if not os.path.isfile(test_pred_path):
        checkpoint_path = glob.glob( model_folder + '/weights.' + str(epochs).zfill(2) + '*.hdf5')[0]
        model = cnn()
        print('loading weights for model {}...'.format(model_id))
        model.load_weights(checkpoint_path)
        print('inference...')
        test_pred = model.predict(x_test)
        np.save(test_pred_path, test_pred)

In [13]:
sample_model_id = 17
y_pred_sample = np.load(os.path.join(output_baseline_folder, str(sample_model_id), 'test_pred.npy'))
shape = np.shape(y_pred_sample)
all_test_preds = np.zeros((n_models, *shape))

In [14]:
d = []
for index, model_id in enumerate(range(17, 17+n_models)):
    y_pred = np.load(os.path.join(output_baseline_folder, str(model_id), 'test_pred.npy'))
    all_test_preds[index] = y_pred
    nll = log_loss(y_test, y_pred)
    error = classification_error(y_test, y_pred)
    br = brier_multi(y_test, y_pred)
    calib_erros = CalibrationErrors(y_test, y_pred, bin_size=1 / 20., min_samples=0)
    _, _, ece, mce, _ = calib_erros.calculate_calibration_errors()
    d.append(OrderedDict({"model":model_id, "nll": nll, "brier": br, "ece": ece, "mce":mce, "classification error": error}))
df_baselines = pd.DataFrame(d)

In [15]:
df_baselines

,model,nll,brier,ece,mce,classification error
0,17,0.036594,0.014640,0.507882,47.358504,0.90
1,18,0.036924,0.016234,0.570997,32.696912,0.98
2,19,0.029432,0.012960,0.465574,68.628386,0.81
3,20,0.040010,0.016554,0.588962,47.255030,1.02
4,21,0.032994,0.014503,0.461238,36.594246,0.92
5,22,0.032003,0.013860,0.487419,57.566917,0.85
6,23,0.043686,0.020463,0.628043,38.309494,1.36
7,24,0.032765,0.014113,0.465772,47.748852,0.89
8,25,0.043138,0.017843,0.630829,66.212824,1.15
9,26,0.032941,0.013969,0.453493,42.323929,0.86


In [16]:
print('Baseline Results (test set)'.upper())
print('-'*100)
metrics = ['nll', 'brier', 'ece', 'mce', 'classification error']
for metric in metrics:
    print('{0}'.format(metric.upper()))
    print('{0:.3f} \u00B1 {1:.2f}'.format(np.mean(df_baselines[metric]), np.std(df_baselines[metric])))
    print('-'*20)

BASELINE RESULTS (TEST SET)
----------------------------------------------------------------------------------------------------
NLL
0.036 ± 0.00
--------------------
BRIER
0.016 ± 0.00
--------------------
ECE
0.517 ± 0.07
--------------------
MCE
48.253 ± 13.45
--------------------
CLASSIFICATION ERROR
0.990 ± 0.13
--------------------


# Deep Ensembles

## TODO: Increase number of models and do bootstrapping for DE to provide std...

In [17]:
n_bootstrap = 100
m_deep_ensemble = 10

In [18]:
d = []
for i in range(n_bootstrap):
    model_indices = np.random.choice(np.shape(all_test_preds)[0], size=m_deep_ensemble, replace=False)
    y_pred = np.mean(np.stack(all_test_preds[model_indices], axis=0), axis=0)
    nll = log_loss(y_test, y_pred)
    error = classification_error(y_test, y_pred)
    br = brier_multi(y_test, y_pred)
    calib_erros = CalibrationErrors(y_test, y_pred, bin_size=1 / 20., min_samples=0)
    _, _, ece, mce, _ = calib_erros.calculate_calibration_errors()
    d.append(OrderedDict({"model":model_id, "nll": nll, "brier": br, "ece": ece, "mce":mce, "classification error": error}))
df_deep_ensembles = pd.DataFrame(d)

In [19]:
df_deep_ensembles

,model,nll,brier,ece,mce,classification error
0,41,0.020074,0.010073,0.246266,13.732367,0.68
1,41,0.020573,0.010293,0.293423,35.811099,0.67
2,41,0.020677,0.010444,0.268155,31.345645,0.67
3,41,0.020650,0.010076,0.288076,31.317680,0.72
4,41,0.020548,0.010576,0.231962,69.357126,0.72
...,...,...,...,...,...,...
95,41,0.020564,0.010060,0.258543,36.640974,0.68
96,41,0.020186,0.010072,0.225385,55.817862,0.62
97,41,0.021173,0.010597,0.212465,69.458918,0.67
98,41,0.021475,0.010398,0.369450,32.932890,0.62


In [20]:
print('Deep Ensemble Results (test set)'.upper())
print('-'*100)
metrics = ['nll', 'brier', 'ece', 'mce', 'classification error']
for metric in metrics:
    print('{0}'.format(metric.upper()))
    print('{0:.3f} \u00B1 {1:.2f}'.format(np.mean(df_deep_ensembles[metric]), np.std(df_deep_ensembles[metric])))
    print('-'*20)

DEEP ENSEMBLE RESULTS (TEST SET)
----------------------------------------------------------------------------------------------------
NLL
0.021 ± 0.00
--------------------
BRIER
0.010 ± 0.00
--------------------
ECE
0.288 ± 0.05
--------------------
MCE
44.856 ± 17.03
--------------------
CLASSIFICATION ERROR
0.661 ± 0.03
--------------------


# Temperature Scaling

In [21]:
from scipy.optimize import minimize
def softmax_t(y_logit, t):
    return np.exp(y_logit/t)/np.sum(np.exp(y_logit/t), axis=-1)[:, np.newaxis]


def ll_t(t):
    y_val_pred = model.predict(x_val)
    y_pred_base_logit = np.log(y_val_pred)
    y_pred_temp = softmax_t(y_pred_base_logit, t)
    ll = log_loss(y_val, y_pred_temp)
    return ll

In [22]:
optimal_temps = []
optimal_temps_path = os.path.join(output_baseline_folder, 'optimal_temps.csv')
if not os.path.isfile(optimal_temps_path):
# if True:
    for index, model_id in enumerate(range(17, 17+ n_models)):
        print('finding optimal sigma for model {}...'.format(model_id))
        model_folder = os.path.join(output_baseline_folder, str(model_id))
        checkpoint_path = glob.glob( model_folder + '/weights.' + str(epochs).zfill(2) + '*.hdf5')[0]
        model = cnn()
        print('loading weights...')
        model.load_weights(checkpoint_path)
        print('running optimization...')
        xopt = minimize(ll_t, 1, method='bfgs', options={'disp': 1})
        optimal_temp = xopt['x'][0]
        print('-'*100)
        print(model_id, optimal_temp)
        optimal_temps.append(optimal_temp)
    pd.DataFrame(optimal_temps, columns=['optimal temp']).to_csv(optimal_temps_path, index=False)
else:
    optimal_temps = pd.read_csv(optimal_temps_path).values
print(optimal_temps)

[[1.55223098]
 [1.53834358]
 [1.4973783 ]
 [1.45628722]
 [1.46255998]
 [1.54153129]
 [1.3807363 ]
 [1.56610639]
 [1.48526209]
 [1.47919712]
 [1.54502638]
 [1.40658667]
 [1.52610853]
 [1.54012705]
 [1.47425313]
 [1.4919964 ]
 [1.45173708]
 [1.52414143]
 [1.60419805]
 [1.50656317]
 [1.48144516]
 [1.44980195]
 [1.5177191 ]
 [1.47435017]
 [1.47740948]]


In [23]:
for index, model_id in enumerate(range(17, 17 + n_models)):
    model_folder = os.path.join(output_baseline_folder, str(model_id))
    output_ts_folder = os.path.join(model_folder, 'temp_scaled')
    if not os.path.isdir(output_ts_folder):
        os.mkdir(output_ts_folder)
    y_pred = np.load(os.path.join(model_folder, 'test_pred.npy'))
    optimal_temp = optimal_temps[index]
    #
    y_pred_logit = np.log(y_pred)
    y_pred_ts = softmax_t(y_pred_logit, t=optimal_temp)
    #
    np.save(os.path.join(output_ts_folder, 'test_pred_ts.npy'), y_pred_ts)

In [24]:
d = list()
for index, model_id in enumerate(range(17, 17 + n_models)):
    model_ppe_folder= os.path.join(output_baseline_folder, str(model_id),  'temp_scaled')
    y_pred = np.load(os.path.join(model_ppe_folder, 'test_pred_ts.npy'))
    nll = log_loss(y_test, y_pred)
    error = classification_error(y_test, y_pred)
    br = brier_multi(y_test, y_pred)
    calib_erros = CalibrationErrors(y_test, y_pred, bin_size=1 / 20., min_samples=0)
    _, _, ece, mce, _ = calib_erros.calculate_calibration_errors()
    d.append(OrderedDict({"model":model_id, "nll": nll, "brier": br, "ece": ece, "mce":mce, "classification error": error}))
df_ts= pd.DataFrame(d)

In [25]:
df_ts

,model,nll,brier,ece,mce,classification error
0,17,0.031236,0.014191,0.227599,36.864736,0.90
1,18,0.032029,0.015551,0.276222,46.762324,0.98
2,19,0.025831,0.012337,0.215158,73.471432,0.81
3,20,0.034956,0.016178,0.199902,43.026470,1.02
4,21,0.029568,0.013980,0.227437,42.098110,0.92
5,22,0.028597,0.013493,0.192260,66.670550,0.85
6,23,0.040633,0.019859,0.339063,35.075896,1.36
7,24,0.029155,0.013422,0.225140,33.657393,0.89
8,25,0.036750,0.017061,0.248283,71.363016,1.15
9,26,0.029013,0.013458,0.167293,37.653620,0.86


In [26]:
print('Temperature Scaling Results (test set)'.upper())
print('-'*100)
metrics = ['nll', 'brier', 'ece', 'mce', 'classification error']
for metric in metrics:
    print('{0}'.format(metric.upper()))
    print('{0:.3f} \u00B1 {1:.2f}'.format(np.mean(df_ts[metric]), np.std(df_ts[metric])))
    print('-'*20)

TEMPERATURE SCALING RESULTS (TEST SET)
----------------------------------------------------------------------------------------------------
NLL
0.032 ± 0.00
--------------------
BRIER
0.015 ± 0.00
--------------------
ECE
0.259 ± 0.06
--------------------
MCE
45.456 ± 15.53
--------------------
CLASSIFICATION ERROR
0.990 ± 0.13
--------------------


## PPE

In [27]:
class PPE:
    def __init__(self, model, n_ensemble):
        self.model = model
        self.n_ensemble = n_ensemble
        self.original_weights = model.get_weights()
        
    def perturb(self, sigma):
        self.model.set_weights(self.original_weights)
        weights = self.model.get_weights()
        for index2, w in enumerate(weights):
            shape = w.shape
            if len(shape) >= 2  and shape[-1] > 32:
                if len(shape) == 4:
                    noise = np.random.normal(0, sigma, (w.shape[0], w.shape[1], w.shape[2], w.shape[3]))
                else:
                    noise = np.random.normal(0, sigma, (w.shape[0], w.shape[1]))
                weights[index2] = weights[index2] + noise
        self.model.set_weights(weights)
    
    def ensemble_pred(self, x, sigma):
        y_pred = self.model.predict(x)
        y_pred_ensemble = np.zeros((self.n_ensemble, *y_pred.shape))
        for seed_index, seed in enumerate(range(17, 17 + self.n_ensemble)):
            np.random.seed(seed)
            self.perturb(sigma)
            y_pred = self.model.predict(x)
            y_pred_ensemble[seed_index] = y_pred
        self.model.set_weights(self.original_weights)
        return np.mean(y_pred_ensemble, axis=0)

In [28]:
def golden_search(model, sigma_lower=0, sigma_upper=0.1, n_ppe_ensemble=5, n_iterations=10):
    sigmas_log = list()
    nlls_log = list()
    nlls = dict()
    PHI = (np.sqrt(5) - 1) / 2
    for iteration in range(n_iterations):
        print('iteration {} {} {} '.format(iteration, sigma_lower, sigma_upper) + '-'* 10)
        sigma_1 = sigma_lower + (sigma_upper - sigma_lower) * PHI
        sigma_2 = sigma_lower + (sigma_1 - sigma_lower) * PHI
        sigmas_log.append(OrderedDict({"iteration": iteration, "sigma_l": sigma_lower,
                                       "sigma_2": sigma_2, "sigma_1": sigma_1,
                                       "sigma_h": sigma_upper}))
        for sigma in [sigma_1, sigma_2]:
            if sigma not in nlls.keys():
                ppe = PPE(model, n_ensemble=n_ppe_ensemble)
                y_val_ppe = ppe.ensemble_pred(x_val, sigma)
                nll = log_loss(y_val, y_val_ppe)
                nlls[sigma] = nll
                print(sigma, nll)
                nlls_log.append(OrderedDict({"iteration": iteration, "sigma": sigma, "nll": nll}))
            else:
                print('will skip for {0:6f}'.format(sigma))
        if nlls[sigma_1] < nlls[sigma_2]:
            sigma_lower = sigma_2
            print('updated lower side...')
        else:
            sigma_upper = sigma_1
            print('updated upper side...')
    sigma_optimal = (sigmas_log[-1]['sigma_l'] + sigmas_log[-1]['sigma_h'])/2
    return sigma_optimal

In [29]:
optimal_sigmas = []
optimal_sigmas_path = os.path.join(output_baseline_folder, 'golden_optimization_simgas.csv')
if not os.path.isfile(optimal_sigmas_path):
    for index, model_id in enumerate(range(17, 17 + n_models)):
        print('finding optimal sigma for model {}...'.format(model_id))
        model_folder = os.path.join(output_baseline_folder, str(model_id))
        checkpoint_path = glob.glob( model_folder + '/weights.' + str(epochs).zfill(2) + '*.hdf5')[0]
        model = cnn()
        print('loading weights...')
        model.load_weights(checkpoint_path)
        print('running optimization...')
        optimal_sigma = golden_search(model)
        optimal_sigmas.append(optimal_sigma)
    df = pd.DataFrame(optimal_sigmas, columns=['optimal sigma'])
    df.to_csv(optimal_sigmas_path, index=False)
else:
    df = pd.read_csv(optimal_sigmas_path)
optimal_sigmas = df['optimal sigma'].values
print(optimal_sigmas)

[0.01656315 0.01818927 0.01261646 0.01524758 0.01311896 0.01737621
 0.01606065 0.01443452 0.01606065 0.01443452 0.01393202 0.00410197
 0.01737621 0.01393202 0.01737621 0.01656315 0.01656315 0.01524758
 0.00917228 0.02082039 0.01656315 0.01524758 0.01869177 0.01443452
 0.01524758]


In [30]:
print(optimal_sigmas)

[0.01656315 0.01818927 0.01261646 0.01524758 0.01311896 0.01737621
 0.01606065 0.01443452 0.01606065 0.01443452 0.01393202 0.00410197
 0.01737621 0.01393202 0.01737621 0.01656315 0.01656315 0.01524758
 0.00917228 0.02082039 0.01656315 0.01524758 0.01869177 0.01443452
 0.01524758]


In [31]:
n_ppe_ensemble = 25
for index, model_id in enumerate(range(17, 17+n_models)):
    model_folder = os.path.join(output_baseline_folder, str(model_id))
    output_ppe_folder = os.path.join(model_folder, 'ppe')
    if not os.path.isdir(output_ppe_folder):
        os.mkdir(output_ppe_folder)
    test_pred_path = os.path.join(output_ppe_folder, 'test_pred_ppe.npy')
    if not os.path.isfile(test_pred_path):
    # if True:
        checkpoint_path = glob.glob(model_folder + '/weights.' + str(epochs).zfill(2) + '*.hdf5')[0]
        model = cnn()
        print('loading weights for model {}...'.format(model_id))
        model.load_weights(checkpoint_path)
        optimal_sigma = optimal_sigmas[index]
        ppe = PPE(model, n_ensemble=n_ppe_ensemble)
        y_pred = ppe.ensemble_pred(x_test, optimal_sigma)
        np.save(test_pred_path, y_pred)
    else:
        continue

In [32]:
d = list()
for index, model_id in enumerate(range(17, 17 + n_models)):
    model_ppe_folder= os.path.join(output_baseline_folder, str(model_id),  'ppe')
    y_pred = np.load(os.path.join(model_ppe_folder, 'test_pred_ppe.npy'))
    nll = log_loss(y_test, y_pred)
    error = classification_error(y_test, y_pred)
    br = brier_multi(y_test, y_pred)
    calib_erros = CalibrationErrors(y_test, y_pred, bin_size=1 / 20., min_samples=0)
    _, _, ece, mce, _ = calib_erros.calculate_calibration_errors()
    d.append(OrderedDict({"model":model_id, "nll": nll, "brier": br, "ece": ece, "mce":mce, "classification error": error}))
df_ppe = pd.DataFrame(d)

In [33]:
df_ppe

,model,nll,brier,ece,mce,classification error
0,17,0.032864,0.014360,0.407820,26.800570,0.91
1,18,0.033854,0.016187,0.383372,17.783421,1.04
2,19,0.026642,0.012424,0.364157,67.611680,0.82
3,20,0.038028,0.016371,0.484884,52.044736,1.00
4,21,0.031545,0.014283,0.395764,63.523687,0.94
5,22,0.029846,0.013624,0.256253,70.088703,0.81
6,23,0.040455,0.019717,0.303081,27.208157,1.32
7,24,0.030986,0.013966,0.319389,26.643641,0.86
8,25,0.039387,0.016895,0.399911,70.433485,1.07
9,26,0.030352,0.013360,0.304104,30.806845,0.86


In [34]:
print('PPE Results (test set)'.upper())
print('-'*100)
metrics = ['nll', 'brier', 'ece', 'mce', 'classification error']
for metric in metrics:
    print('{0}'.format(metric.upper()))
    print('{0:.3f} \u00B1 {1:.2f}'.format(np.mean(df_ppe[metric]), np.std(df_ppe[metric])))
    print('-'*20)

PPE RESULTS (TEST SET)
----------------------------------------------------------------------------------------------------
NLL
0.034 ± 0.00
--------------------
BRIER
0.015 ± 0.00
--------------------
ECE
0.366 ± 0.08
--------------------
MCE
42.119 ± 18.98
--------------------
CLASSIFICATION ERROR
0.990 ± 0.12
--------------------


# MC Dropout

In [35]:
def cnn_mcdo(num_classes=10, dropout=True, level=0.5):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(BatchNormalization(momentum=.9))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    if dropout:
        model.add(Lambda(lambda x: K.dropout(x, level=level)))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    if dropout:
        model.add(Lambda(lambda x: K.dropout(x, level=level)))
    
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization(momentum=.9))
    if dropout:
        model.add(Lambda(lambda x: K.dropout(x, level=level)))
    
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [36]:
learning_rate = 5e-4
#
output_baseline_dropout_folder = os.path.join(output_folder, 'baselines_dropout')
if not os.path.isdir(output_baseline_dropout_folder):
    os.makedirs(output_baseline_dropout_folder)

In [37]:
for i in range(n_models):
    seed = i+17
    # change the random state
    np.random.seed(seed)
    # creating output folder for each of the baseline models
    output_model_folder = os.path.join(output_baseline_dropout_folder, str(seed))
    if not os.path.isdir(output_model_folder):
        os.mkdir(output_model_folder)
        # callbacks for training
        callbacks = list()
        log = CSVLogger(os.path.join(output_model_folder, 'log.csv'))
        callbacks.append(log)
        checkpoint_path = os.path.join(output_model_folder, 'weights.{epoch:02d}.hdf5')
        checkpoint = ModelCheckpoint(checkpoint_path, period=1, save_best_only=False, verbose=True)
        callbacks.append(checkpoint)
        # initialize and compile the model
        model = cnn_mcdo()
        model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate), metrics=['accuracy'])
        # train
        model.fit(x_train, y_train, batch_size=128, epochs=15, verbose=1, callbacks=callbacks, validation_data=(x_val, y_val), shuffle=True)

In [ ]:
for i in range(17, 17+n_models):
    log = pd.read_csv(os.path.join(output_baseline_dropout_folder, str(i), 'log.csv'))
    plt.figure(figsize=(8,2),dpi=100)
    plt.subplots_adjust(wspace=0.5)
    ax1 = plt.subplot(121)
    ax2 = plt.subplot(122)
    for metric_type in ['', 'val_']:
        for key in log.keys():
            if 'loss' in key and 'val' not in key:
                p = ax1.plot(log[metric_type+ key], label=metric_type + key+str(i), linewidth=1)
                c = p[-1].get_color()
                ax1.plot(np.argmin(log[metric_type+key]), np.amin(log[metric_type+key]), 
                         '*', markersize=10, alpha=0.5, color = c)
                ax1.set_title('min val NLL: {0:.3f}'.format(np.amin(log['val_' + key])))
                ax1.set_ylabel('NLL')
                ax1.set_xlabel('epochs')
            if 'acc' in key and 'val' not in key:
                p = ax2.plot(log[metric_type + key], label=metric_type + key+str(i), linewidth=1)
                c = p[-1].get_color()
                ax2.plot(np.argmax(log[metric_type + key]), np.amax(log[metric_type + key]), 
                         '*', markersize=10, alpha=0.5, color=c)
                ax2.set_title('max val acc: {0:.2f}'.format(np.amax(log['val_' + key]*100)))
                ax2.set_ylabel('accuracy')
                ax2.set_xlabel('epochs')
    ax1.grid()
    ax2.grid()
    ax1.legend()
    ax2.legend()

In [39]:
for index, model_id in enumerate(range(17, 17+n_models)):
    model_folder = os.path.join(output_baseline_dropout_folder, str(model_id))
    test_pred_path = os.path.join(model_folder, 'test_pred.npy')
    # if True:
    if not os.path.isfile(test_pred_path):
        checkpoint_path = glob.glob(model_folder + '/weights.' + str(epochs).zfill(2) + '*.hdf5')[0]
        model = cnn_mcdo(dropout=False)
        print('loading weights for model {}...'.format(model_id))
        model.load_weights(checkpoint_path)
        print('inference...')
        test_pred = model.predict(x_test)
        np.save(test_pred_path, test_pred)

In [40]:
sample_model_id = 17
y_pred_sample = np.load(os.path.join(output_baseline_folder, str(sample_model_id), 'test_pred.npy'))
shape = np.shape(y_pred_sample)
all_test_preds = np.zeros((n_models, *shape))

In [41]:
d = []
for index, model_id in enumerate(range(17, 17 + n_models)):
    y_pred = np.load(os.path.join(output_baseline_dropout_folder, str(model_id), 'test_pred.npy'))
    all_test_preds[index] = y_pred
    nll = log_loss(y_test, y_pred)
    error = classification_error(y_test, y_pred)
    br = brier_multi(y_test, y_pred)
    calib_erros = CalibrationErrors(y_test, y_pred, bin_size=1 / 20., min_samples=0)
    _, _, ece, mce, _ = calib_erros.calculate_calibration_errors()
    d.append(OrderedDict({"model":model_id, "nll": nll, "brier": br, "ece": ece, "mce":mce, "classification error": error}))
df_baselines_do = pd.DataFrame(d)

In [42]:
print('Baseline Dropout Results (test set)'.upper())
print('-'*100)
metrics = ['nll', 'brier', 'ece', 'mce', 'classification error']
for metric in metrics:
    print('{0}'.format(metric.upper()))
    print('{0:.3f} \u00B1 {1:.2f}'.format(np.mean(df_baselines_do[metric]), np.std(df_baselines_do[metric])))
    print('-'*20)

BASELINE DROPOUT RESULTS (TEST SET)
----------------------------------------------------------------------------------------------------
NLL
0.046 ± 0.03
--------------------
BRIER
0.019 ± 0.01
--------------------
ECE
0.589 ± 0.29
--------------------
MCE
47.715 ± 13.74
--------------------
CLASSIFICATION ERROR
1.179 ± 0.33
--------------------


In [43]:
n_mcdo_ensemble = 25
for index, model_id in enumerate(range(17, 17+n_models)):
    model_folder = os.path.join(output_baseline_dropout_folder, str(model_id))
    output_mcdo_folder = os.path.join(model_folder, 'mcdo')
    test_pred_path = os.path.join(output_mcdo_folder, 'test_pred_mcdo.npy')
    # if True:
    if not os.path.isfile(test_pred_path):
        if not os.path.isdir(output_mcdo_folder):
            os.mkdir(output_mcdo_folder)
        checkpoint_path = glob.glob(model_folder + '/weights.' + str(epochs).zfill(2) + '*.hdf5')[0]
        model = cnn_mcdo()
        print('loading weights for model {}...'.format(model_id))
        model.load_weights(checkpoint_path)
        y_test_pred_ensemble = np.zeros((n_mcdo_ensemble, *y_test.shape))
        for index2, seed in enumerate(range(17, 17 + n_mcdo_ensemble)):
            np.random.seed(seed)
            y_test_pred_dropout = model.predict(x_test, verbose=1)
            y_test_pred_ensemble[index2] = y_test_pred_dropout
        y_pred = np.mean(y_test_pred_ensemble, axis=0)
        np.save(test_pred_path, y_pred)

In [44]:
d = list()
for index, model_id in enumerate(range(17, 17 + n_models)):
    model_mcdo_folder= os.path.join(output_baseline_dropout_folder, str(model_id),  'mcdo')
    y_pred = np.load(os.path.join(model_mcdo_folder, 'test_pred_mcdo.npy'))
    nll = log_loss(y_test, y_pred)
    error = classification_error(y_test, y_pred)
    br = brier_multi(y_test, y_pred)
    calib_erros = CalibrationErrors(y_test, y_pred, bin_size=1 / 20., min_samples=0)
    _, _, ece, mce, _ = calib_erros.calculate_calibration_errors()
    d.append(OrderedDict({"model":model_id, "nll": nll, "brier": br, "ece": ece, "mce":mce, "classification error": error}))
df_mcdo= pd.DataFrame(d)

In [45]:
df_mcdo

,model,nll,brier,ece,mce,classification error
0,17,0.030228,0.013708,0.786098,39.461466,0.82
1,18,0.031691,0.014284,0.817432,71.709560,0.89
2,19,0.031044,0.013723,0.728056,66.231902,0.82
3,20,0.032092,0.013702,0.866675,67.469836,0.82
4,21,0.033586,0.014869,0.785937,18.006981,0.92
5,22,0.031600,0.014381,0.819364,41.477634,0.92
6,23,0.028915,0.012811,0.788859,37.256537,0.83
7,24,0.030110,0.013116,0.930001,46.525903,0.76
8,25,0.032393,0.014452,0.760403,33.391998,0.85
9,26,0.032234,0.014276,0.880209,66.729956,0.82


In [46]:
print('MCDO Results (test set)'.upper())
print('-'*100)
metrics = ['nll', 'brier', 'ece', 'mce', 'classification error']
for metric in metrics:
    print('{0}'.format(metric.upper()))
    print('{0:.3f} \u00B1 {1:.2f}'.format(np.mean(df_mcdo[metric]), np.std(df_mcdo[metric])))
    print('-'*20)

MCDO RESULTS (TEST SET)
----------------------------------------------------------------------------------------------------
NLL
0.031 ± 0.00
--------------------
BRIER
0.014 ± 0.00
--------------------
ECE
0.832 ± 0.06
--------------------
MCE
46.252 ± 18.90
--------------------
CLASSIFICATION ERROR
0.842 ± 0.06
--------------------


In [47]:
dfs = [df_baselines,
df_ppe,
df_ts,
df_mcdo,
df_deep_ensembles
      ]

In [48]:
metrics = ['nll', 'brier', 'ece', 'mce', 'classification error']
for metric in metrics:
    print('% {}'.format(metric))
    for index, df in enumerate(dfs):
        if index != len(dfs)-1:
            print('{0:.3f} $\pm$ {1:.2f} &'.format(np.mean(df[metric]), np.std(df[metric])))
        else:
            print('{0:.3f} $\pm$ {1:.2f} \\\\'.format(np.mean(df[metric]), np.std(df[metric])))
    print('-'*100)

% nll
0.036 $\pm$ 0.00 &
0.034 $\pm$ 0.00 &
0.032 $\pm$ 0.00 &
0.031 $\pm$ 0.00 &
0.021 $\pm$ 0.00 \\
----------------------------------------------------------------------------------------------------
% brier
0.016 $\pm$ 0.00 &
0.015 $\pm$ 0.00 &
0.015 $\pm$ 0.00 &
0.014 $\pm$ 0.00 &
0.010 $\pm$ 0.00 \\
----------------------------------------------------------------------------------------------------
% ece
0.517 $\pm$ 0.07 &
0.366 $\pm$ 0.08 &
0.259 $\pm$ 0.06 &
0.832 $\pm$ 0.06 &
0.288 $\pm$ 0.05 \\
----------------------------------------------------------------------------------------------------
% mce
48.253 $\pm$ 13.45 &
42.119 $\pm$ 18.98 &
45.456 $\pm$ 15.53 &
46.252 $\pm$ 18.90 &
44.856 $\pm$ 17.03 \\
----------------------------------------------------------------------------------------------------
% classification error
0.990 $\pm$ 0.13 &
0.990 $\pm$ 0.12 &
0.990 $\pm$ 0.13 &
0.842 $\pm$ 0.06 &
0.661 $\pm$ 0.03 \\
---------------------------------------------------------